# Operations Rehearsal for Commissioning #4 June 2024 Consolidated Database Demo

Tutorial notebook for using the consolidated database using LSSTComCam simulated datastream from the Operations Rehearsal for Commissioning #4 in June 2024. This notebook uses example queries developed by Eric Neilsen for a prototype observation night summary notebook in Times Square.

Last verified to run: 26 June 2024

LSST Science Pipelines Version: `w_2024_25`

Consolidated database usage documentation: https://confluence.lsstcorp.org/display/~ktl/ConsDB+Usage

Consolidated database Schema: https://dm.lsst.org/sdm_schemas/browser/cdb_lsstcomcamsim.html (ls.st/schema)

In [ ]:
!eups list -s | grep lsst_distrib

In [ ]:
import pandas as pd

In [ ]:
import os
os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
print(client.schema())

In [ ]:
client.schema()

In [ ]:
client.schema("lsstcomcamsim")

In [ ]:
client.schema("lsstcomcamsim", "cdb_lsstcomcamsim.visit1_quicklook")

In [ ]:
instrument = 'lsstcomcamsim'
day_obs = '2024-06-27'
day_obs_int = int(day_obs.replace('-', ''))

visits_query = f'''
    SELECT * FROM cdb_{instrument}.exposure
        WHERE obs_start_mjd IS NOT NULL
        AND s_ra IS NOT NULL
        AND s_dec IS NOT NULL
        AND sky_rotation IS NOT NULL
        AND ((band IS NOT NULL) OR (physical_filter IS NOT NULL))
        AND day_obs = {day_obs_int}
'''

#visits_query = f'''
#    SELECT * FROM cdb_{instrument}.exposure
#        WHERE day_obs = {day_obs_int}
#'''

In [ ]:
visits = client.query(visits_query).to_pandas()

In [ ]:
visits.columns

In [ ]:
visits

In [ ]:
exposure_opsimdb_map = {
        'obs_start_mjd': 'observationStartMJD',
        'obs_start': 'start_date',
        's_ra': 'fieldRA',
        's_dec': 'fieldDec',
        'sky_rotation': 'rotSkyPos',
        'band': 'filter',
        'airmass': 'airmass',
        'altitude': 'altitude',
        'azimuth': 'azimuth',
        'exp_time': 'visitExposureTime'
    }

visits.rename(exposure_opsimdb_map, axis='columns', inplace=True)

In [ ]:
missing_filter = visits['filter'].isna()
visits.loc[missing_filter, 'filter'] = visits.loc[missing_filter, 'physical_filter'].str.get(0)

In [ ]:
if len(visits):
    displayed_columns = [
        "start_date",
        "seq_num",
        "fieldRA",
        "fieldDec",
        "filter",
        "visitExposureTime",
#        "numExposures",
#        "t_eff",
#        "skyBrightness",
#        "seeingFwhmEff",
#        "cloud",
#        "note",
    ]
    displayed_visits_df = visits.loc[:, displayed_columns]
    with pd.option_context("display.max_rows", 2000):
        display(displayed_visits_df)
else:
    print("No visits")
